<a href="https://colab.research.google.com/github/barry80417/Prediction-of-quality-of-Wine/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C%EF%BC%8302%EF%BC%9A%E7%B4%85%E9%85%92%E5%93%81%E8%B3%AA%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境準備

In [ ]:
from google.colab import files
#上傳CSV
uploaded = files.upload()
import io
wine = pd.read_csv(io.BytesIO(uploaded['winequality-red.csv']))

In [73]:
import pandas as pd
import numpy as np
wine = pd.read_csv("https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv")

1 - fixed acidity:固定酸度

2 - volatile acidity：揮發性酸度

3 - citric acid：檸檬酸

4 - residual sugar：殘糖

5 - chlorides：氯化物

6 - free sulfur dioxide：游離二氧化硫

7 - total sulfur dioxide：總二氧化硫

8 - density：密度

9 - pH：酸鹼度

10 - sulphates：硫酸鹽

11 - alcohol Output variable (based on sensory data):酒精輸出變量

12 - quality (score between 0 and 10)：品質

## 檢視有無缺失值

In [ ]:
wine.count()

## 敘述性分析

In [71]:
wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


## 看各個資料關聯性

In [ ]:
#@title train data relation
#看訓練資料中是否有所關聯
import matplotlib.pyplot as plt
import seaborn as sns
corr = wine.corr()
cmap = sns.diverging_palette(200, 0, as_cmap = True)
sns.heatmap(
    corr,
    cmap=cmap,
    square = False,
    cbar_kws = {'shrink': .9}, #顏色條大小
    ax = plt.subplots(figsize =(12,10))[1],
    annot = True,
    annot_kws = {'fontsize':12} #字體大小
)

## 參考 https://dsgeeksdotcom.wordpress.com/2020/04/29/predicting-the-quality-of-red-wine-using-python/ 查取outlier盒形圖

In [ ]:
fig, ax = plt.subplots(ncols = 5,nrows = 2, figsize=(15,5))
ax = ax.flatten()
index = 0
for i in wine.columns:
  if i != 'quality':
    sns.boxplot(y = i, data=wine, ax=ax[index])
    index +=1
plt.tinght_layout(pad = 0.4)
plt.show()

## 資料正規化
1.自己調整

In [55]:
from sklearn import preprocessing
minmax = preprocessing.MinMaxScaler() #Max-Min正規化
wine['fixed acidity'] = minmax.fit_transform(wine['fixed acidity'].values.reshape(-1,1))
wine = wine.drop(['citric acid','free sulfur dioxide'],axis = 1)
wine['residual sugar'] = minmax.fit_transform(wine['residual sugar'].values.reshape(-1,1))
wine['total sulfur dioxide'] = minmax.fit_transform(wine['total sulfur dioxide'].values.reshape(-1,1))

2.統一調整正規化

In [74]:
# 依據Santa Rosa article(http://srjcstaff.santarosa.edu/~jhenderson/SO2.pdf)，quality越高，游離二氧化硫濃度越高
wine["mso2"] = wine['free sulfur dioxide']/(1+10**wine['pH'] - 1.81)
wine = wine.drop(['citric acid','free sulfur dioxide'],axis = 1)

from sklearn.preprocessing import StandardScaler
# normalization，資料正規化
scaler = StandardScaler()
feature_columns = wine.columns.difference(["quality"])
wine[feature_columns] = scaler.fit_transform(wine[feature_columns])

In [10]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   residual sugar        1599 non-null   float64
 3   chlorides             1599 non-null   float64
 4   total sulfur dioxide  1599 non-null   float64
 5   density               1599 non-null   float64
 6   pH                    1599 non-null   float64
 7   sulphates             1599 non-null   float64
 8   alcohol               1599 non-null   float64
 9   quality               1599 non-null   int64  
 10  mso2                  1599 non-null   float64
 11  qual                  1599 non-null   object 
dtypes: float64(10), int64(1), object(1)
memory usage: 150.0+ KB


## quality區分好壞
1.多切一個欄位

In [67]:
wine.loc[wine['quality']>=7,'qual']='good'
wine.loc[wine['quality']<7,'qual']='bad'
from sklearn.preprocessing import LabelEncoder
laberencoder = LabelEncoder()
wine['qual'] = laberencoder.fit_transform(wine['qual'])

2.改變原有欄位

In [75]:
# 將quality二分為"good" and "bad"
bins = (0, 6.5, 10)
group_names = ['bad', 'good']
wine['quality'] = pd.cut(wine['quality'], bins = bins, labels = group_names)
from sklearn.preprocessing import LabelEncoder
laberencoder = LabelEncoder()
wine['quality'] = laberencoder.fit_transform(wine['quality'])

## 用圖表看品質是否資料不平衡

In [ ]:
plt.figure(figsize = (10,8))
wine['qual'].value_counts().plot(kind = 'pie',colors = ['lightcoral','skyblue'],autopct = '%1.2f%%')

## 資料定義



In [76]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


X = wine.drop(['quality'], axis = 1)
y = wine['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1,random_state = 43, stratify=y_train)

## 看訓練及內資料數

In [40]:
display(X_train.shape)
display(X_test.shape)

(1151, 10)

(320, 10)

In [42]:
y_train.value_counts()

0    998
1    998
Name: qual, dtype: int64

## 資料不平衡處理

In [41]:
from imblearn.over_sampling import SMOTE
X_train, y_train = SMOTE().fit_resample(X_train,y_train)

## 隨機森林

In [60]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200)
rfc_eval = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 5)
rfc_eval


array([0.93506494, 0.91304348, 0.9       , 0.9       , 0.92173913])

In [61]:
from sklearn.metrics import classification_report,accuracy_score
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)
print(classification_report(y_test,pred_rfc))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94       273
           1       0.69      0.57      0.63        47

    accuracy                           0.90       320
   macro avg       0.81      0.77      0.79       320
weighted avg       0.89      0.90      0.90       320



## KNN模型

In [46]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 2)
knn_eval = cross_val_score(estimator = knn, X = X_train, y = y_train, cv = 5)
knn_eval


array([0.9575    , 0.93483709, 0.92481203, 0.93483709, 0.95989975])

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
knn.fit(X_train, y_train)
pred_knn = knn.predict(X_test)
print(classification_report(y_test,pred_rfc))

## 決策數

In [48]:
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier()
dct_eval = cross_val_score(estimator = dct, X = X_train, y = y_train, cv = 5)
dct_eval

array([0.86      , 0.9122807 , 0.87719298, 0.90225564, 0.91729323])

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
dct.fit(X_train, y_train)
pred_dct = dct.predict(X_test)
print(classification_report(y_test,pred_rfc))

## 向量機

In [78]:
from sklearn.svm import SVC
svc = SVC()
svc_eval = cross_val_score(estimator = svc, X = X_train, y = y_train, cv = 5)
svc_eval

array([0.8671875 , 0.8671875 , 0.8671875 , 0.8671875 , 0.86666667])

## XGBoost

In [81]:
import xgboost as xgb
xgbm = xgb.XGBClassifier()
xgbm_eval = cross_val_score(estimator = xgbm, X = X_train, y = y_train, cv = 5)
xgbm_eval

array([0.8671875 , 0.8984375 , 0.87109375, 0.921875  , 0.89411765])

In [ ]:
import torch


# 創建模型架構
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.hidden1 = torch.nn.Linear(input_size, 64)
        self.hidden2 = torch.nn.Linear(64, 32)
        self.predict = torch.nn.Linear(32, output_size)
        
    def forward(self, x):
        output1 = torch.relu(self.hidden1(x))
        output2 = torch.relu(self.hidden2(output1))
        output = torch.sigmoid(self.predict(output2))
        return output

# 模型、優化器初始化
model = Model(X_test.shape[1], 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.0) # 模型參數優化器
loss_func = torch.nn.BCELoss()


# 資料格式轉成torch專用格式
x_data = torch.tensor(X_train.values, dtype=torch.float32)
y_data = torch.tensor(np.expand_dims(y_train, axis=1), dtype=torch.float32)

x_val_data = torch.tensor(X_val.values, dtype=torch.float32)
y_val_data = torch.tensor(np.expand_dims(y_val, axis=1), dtype=torch.float32)

x_test_data = torch.tensor(X_test.values, dtype=torch.float32)
y_test_data = torch.tensor(np.expand_dims(y_test, axis=1), dtype=torch.float32)


batch_size = 32
num_epochs = 200

training_losses = []
val_losses = []

for epoch in range(num_epochs):
    for i in range(0, len(x_data), batch_size):
        prediction = model(x_data[i:i+batch_size])
        loss = loss_func(prediction, y_data[i:i+batch_size])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss = loss_func(model(x_data), y_data)
    training_losses.append(float(loss))

    y_pred = model(x_val_data)
    val_loss = loss_func(y_pred, y_val_data)
    print("training loss:{}, val loss:{}, val acc:{}".format(float(loss), val_loss, accuracy_score(y_val_data, np.where(y_pred >= 0.5, 1, 0))))
    
    val_losses.append(float(val_loss))

In [78]:
y_pred = model(x_test_data)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       273
           1       0.69      0.47      0.56        47

    accuracy                           0.89       320
   macro avg       0.80      0.72      0.75       320
weighted avg       0.88      0.89      0.88       320



## Keras

In [62]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 18.6 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers

model = tf.keras.models.Sequential([
    layers.Dense(64, name="hidden1", activation="relu"),
    layers.Dense(32, name="hidden2", activation="relu"),
    layers.Dense(1, name="output", activation=tf.nn.sigmoid),
])
optimizer = keras.optimizers.SGD(lr=0.05)
model.compile(optimizer=optimizer, loss=tfa.losses.SigmoidFocalCrossEntropy(), metrics=["Accuracy"])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)

In [80]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94       273
           1       0.69      0.62      0.65        47

    accuracy                           0.90       320
   macro avg       0.81      0.78      0.80       320
weighted avg       0.90      0.90      0.90       320

